## Demo retinanet

### keras-retinanet / examples /ResNet50RetinaNet.ipynb

https://github.com/fizyr/keras-retinanet/blob/master/examples/ResNet50RetinaNet.ipynb

https://github.com/wucng/TensorExpand/blob/master/TensorExpand/Object%20detection/RetinaNet/Exercise/CSV/test_demo.py

## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Load RetinaNet model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
#model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')
#model_path = os.path.join('..', 'snapshots', 'resnet50_csv_sample.h5')
model_path='/dataDisk/myfloder/jupyter/snapshots/resnet50_csv_trans.h5'
# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
# labels_to_names = {0: '三杯茄子', 1: '地瓜葉', 2: '彩色百頁', 
#                    3: '糖醋豬排', 4: '豆豉小排', 5: '金針菇', 6: '香菇'}

OSError: Unable to open file (unable to open file: name = '/dataDisk/myfloder/jupyter/snapshots/resnet50_csv_trans.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# load label to names mapping for visualization purposes
import csv

label_path='../../inputdata/csv/classes_label.csv'

with open(label_path, newline='') as csvfile:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  reader = csv.DictReader(csvfile)
  labels_to_names = {}
  # 以迴圈輸出指定欄位
  for row in reader:
       labels_to_names[int(row['label'])]= row['name']
  print(labels_to_names)

## Run detection on example

In [ ]:
from PIL import ImageFont, ImageDraw, Image

def draw_caption2(image, box, caption):
    """ Draws a caption above the box in an image.
    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    # 字體設定
    fontPath = "/usr/local/lib/python3.5/dist-packages/matplotlib/mpl-data/fonts/ttf/msj.ttf"
    font = ImageFont.truetype(fontPath,20)
    # 將 NumPy 陣列轉為 PIL 影像
    imgPil = Image.fromarray(image)
    
    # 在圖片上加入文字
    draw = ImageDraw.Draw(imgPil)
    draw.text((b[0], b[1] - 10), caption, font = font, fill = (0, 0, 0))
    draw.text((b[0], b[1] - 10), caption, font = font, fill = (255,255,255))
    #cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    #cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
    
    # 將 PIL 影像轉回 NumPy 陣列
    image = np.array(imgPil)
    
    return image

In [ ]:
# 選擇第二隻攝影機
cap = cv2.VideoCapture(0)
# 設定影像的尺寸大小
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

while(True):
    # 從攝影機擷取一張影像
    ret, image = cap.read()
    # copy to draw on
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    # start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    # print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.7:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)

        draw=draw_caption2(draw, b, caption)
        
       
    
    # 設定視窗大小
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame',draw)
    #plt.figure(figsize=(15, 15))
    #plt.axis('off')
    #plt.imshow(draw)
    #plt.show()

    # 若按下 q 鍵則離開迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

# 釋放攝影機
cap.release()
# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows() 

### no apply

In [ ]:
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.5:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.3f}".format(labels_to_names[label], score)
    
    
    draw=draw_caption2(draw, b, caption)
    
    
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()

In [ ]:
# load image
#image = read_image_bgr('000000008021.jpg')
image = read_image_bgr('Plat6_lunch_20181001_20559_NS.jpg')
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.5:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.3f}".format(labels_to_names[label], score)
    
    
    draw_caption(draw, b, caption)
    
    
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()

In [ ]:
caption